# 5.2 参数管理

## 5.2.1 参数访问

In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
X,net(X)

(tensor([[0.7935, 0.4805, 0.0597, 0.1898],
         [0.1353, 0.9337, 0.3565, 0.8407]]),
 tensor([[0.3908],
         [0.4608]], grad_fn=<AddmmBackward0>))

当通过Sequential类定义模型时，我们可以通过索引来访问模型的任意层。

这就像模型是一个列表一样。每层的参数都在其属性中。

如下所示，我们可以检查第二个全连接层的参数。
首先，这个全连接层包含两个参数，分别是该层的权重和偏置,两者都存储为单精度浮点数（float32）。

注意，参数名称允许我们唯一地标识每个参数，即使在包含数百个层的网络中也是如此。

In [2]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.0989,  0.2280,  0.1226,  0.1477,  0.0780,  0.1395,  0.1414, -0.2219]])), ('bias', tensor([0.2100]))])


### 目标参数
注意，每个参数都表示为参数（parameter）类的一个实例。

要对参数执行任何操作，首先我们需要访问底层的数值。

参数是复合的对象，包含值、梯度和额外信息

In [3]:
print(type(net[2].bias)) # 访问参数的类型
print(net[2].bias) # 访问参数
print(net[2].bias.data) # 访问参数的值

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.2100], requires_grad=True)
tensor([0.2100])


In [4]:
# 我们还可以访问每个参数的梯度。
# 由于我们还没有调用这个网络的反向传播，所以参数的梯度处于初始状态。
net[2].weight.grad == None

True

### 一次性访问所有的参数

In [5]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [6]:
# 另外一种访问网格参数的方式
net.state_dict()['2.bias'].data

tensor([0.2100])

### 从嵌套块收集参数

In [7]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

In [8]:
rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X),rgnet

(tensor([[-0.4090],
         [-0.4090]], grad_fn=<AddmmBackward0>),
 Sequential(
   (0): Sequential(
     (block 0): Sequential(
       (0): Linear(in_features=4, out_features=8, bias=True)
       (1): ReLU()
       (2): Linear(in_features=8, out_features=4, bias=True)
       (3): ReLU()
     )
     (block 1): Sequential(
       (0): Linear(in_features=4, out_features=8, bias=True)
       (1): ReLU()
       (2): Linear(in_features=8, out_features=4, bias=True)
       (3): ReLU()
     )
     (block 2): Sequential(
       (0): Linear(in_features=4, out_features=8, bias=True)
       (1): ReLU()
       (2): Linear(in_features=8, out_features=4, bias=True)
       (3): ReLU()
     )
     (block 3): Sequential(
       (0): Linear(in_features=4, out_features=8, bias=True)
       (1): ReLU()
       (2): Linear(in_features=8, out_features=4, bias=True)
       (3): ReLU()
     )
   )
   (1): Linear(in_features=4, out_features=1, bias=True)
 ))

In [9]:
rgnet[0][1][0].bias.data

tensor([ 0.2435, -0.2207,  0.1601,  0.4721,  0.0613, -0.0044,  0.4204, -0.2730])

## 5.2.2 参数初始化

### 内置初始化

In [10]:
# 将所有权重参数初始化为标准差为0.01的高斯随机变量
# 且将偏置参数设置为0
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0156,  0.0021,  0.0112, -0.0036]), tensor(0.))

In [11]:
# 将所有参数初始化为给定的常数（比如1）
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [12]:
# 可以对某些块应用不同的初始化方法。
# 例如，下面我们使用Xavier初始化方法初始化第一层，
# 然后第二层初始化为常量值42。
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)


tensor([ 0.3714, -0.4434,  0.0276, -0.5545])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


### 自定义初始化

In [13]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]


Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-9.4184,  0.0000, -8.8357, -0.0000],
        [ 0.0000,  0.0000, -0.0000,  5.8973]], grad_fn=<SliceBackward0>)

In [14]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000, -7.8357,  1.0000])

## 5.2.3 参数绑定

In [16]:
# 这个例子表明第二层和第三层的参数是绑定的。
# 它们不仅值相等，而且由相同的张量表示。
# 我们需要给共享层一个名称，以便可以引用它的参数。
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
print(net)
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值。
print(net[2].weight.data[0] == net[4].weight.data[0])


Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=8, bias=True)
  (3): ReLU()
  (4): Linear(in_features=8, out_features=8, bias=True)
  (5): ReLU()
  (6): Linear(in_features=8, out_features=1, bias=True)
)
tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


## 5.2.4 小结
* 我们有几种方法可以访问、初始化和绑定模型参数。
* 我们可以使用自定义初始化方法。